In [4]:
#提取石油产业区的实际归属地

import pymysql
import pandas

connection = pymysql.connect(host='47.92.25.70', user='root', password='Wfn031641', db='zhaoshang', charset='utf8')

sql = "select `id`,`com`,`area` from `dic_area`"
df = pandas.read_sql(sql, connection)

name_list = []
id_del = []

for index in df.index:
    name = df.loc[index, 'com']
    id_n = df.loc[index, 'id']
    if name in name_list:
        id_del.append(id_n)
    name_list.append(name)
#print(name_del)
print(len(id_del))

1907


In [5]:
for name_id in id_del:
    connection = pymysql.connect(host='47.92.25.70', user='root', password='Wfn031641', db='zhaoshang', charset='utf8')
    with connection.cursor() as cursor: 
        sql = "delete from `dic_area` where `id` = '{}'".format(str(name_id))
        cursor.execute(sql)
    connection.close()

In [13]:
sql_t = "select `com`,`q_id`,`province`,`city`,`address`,`phone` from `dic_company` where `address` like '%长兴岛经济区%'"
df_t = pandas.read_sql(sql_t, connection)
len(df_t)

11086

In [1]:
#功能整合函数
import pymysql
import pandas
from bs4 import BeautifulSoup
import requests
import numpy as np

#http://www.ip138.com:8080/search.asp?mobile={}&action=mobile.format('13998615738')
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0"
    }

def get_mobile_location(num): #通过电话查归属地
    url = 'http://www.ip138.com:8080/search.asp?mobile={}&action=mobile'.format(str(num))
    web_origin = requests.get(url, headers=header).content
    soup = BeautifulSoup(web_origin, 'html.parser')
    num_form = soup.find_all('td', attrs={'class':'tdc2'})
    num_location = num_form[1].get_text()
    province, city = num_location.split('\xa0')
    return province, city

def get_qid(name): #获取企查查id（库内）
    connection = pymysql.connect(host='47.92.25.70', user='root', password='Wfn031641', db='zhaoshang', charset='utf8')
    with connection.cursor() as cursor: 
        sql = "select `q_id` from `dic_company` where `name` = '{}'".format(name)
        cursor.execute(sql)
        q_id = cursor.fetchone()[0]
    connection.close()
    return q_id

def get_update_df(df): #查询归属地和企查查id后写入df
    df['phone'].fillna('-', inplace=True)
    df['address'].fillna('-', inplace=True)
    df['city'].fillna('市区', inplace=True)
    for index in df.index:
        phone = str(df.loc[index, 'phone'])
        name = df.loc[index, 'com']
        if phone == None:
            continue
        elif '-' in str(phone):
            df.loc[index, 'phone'] = None
            continue
        else:
            try:
                q_id = get_qid(name)
            except:
                print(name, '-->库内无该企业')
            try:
                r_province, r_city = get_mobile_location(phone)
                df.loc[index, 'province_real'] = r_province
                df.loc[index, 'city_real'] = r_city
            except:
                print(index, phone)
                continue
    return df

def get_qichacha_df(dir_qichacha, area_name): #获取企查查上下载表格整合后的df
    df = pandas.read_excel(dir_qichacha)
    df_new = df[['企业名称','省份','城市','地址','电话']]
    df_new.columns = ['com','province','city','address','phone']
    df_new['province_real'] = None
    df_new['city_real'] = None
    df_new['area'] = area_name
    df_new['q_id'] = None
    df_final = get_update_df(df_new)
    return df_final

def update_df(df): #上传至数据库
    for index in df.index:
        row = list(df.loc[index].values)
        connection = pymysql.connect(host='47.92.25.70', user='root', password='Wfn031641', db='zhaoshang', charset='utf8')
        with connection.cursor() as cursor: 
            sql_k = "insert into `dic_area`(`com`,`province`,`city`,`address`,`phone`,`province_real`,`city_real`,`area`,`q_id`) \
            values(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql_k, row)
            connection.commit()
        print(row[0],'finished')
        connection.close()
    print('Done!')
    
    
def main(dir_input, area_name):
    df_q = get_qichacha_df(dir_input, area_name)
    df_u = get_update_df(df_q)
    update_df(df_u)
    
#main('D:/data/qichacha/大连保税区/merge/保税区石化企业.xls')
#main('D:/data/qichacha/舟山保税区/merge/舟山保税区石化企业.xls', '舟山保税区')
#main('D:/data/qichacha/上海自贸区/merge/上海自贸区石化企业.xls', '上海自贸区')
#main('D:/data/qichacha/大连自贸区/大连自贸区石化企业.xls', '大连自贸区')
#main('D:/data/qichacha/淄博高新区/merge/淄博高新区石化企业.xls', '淄博高新区')
#main('D:/data/qichacha/东营经济开发区/merge/东营经济开发区石化企业.xls', '东营经济开发区')
#main('D:/data/qichacha/盘锦辽东湾新区/merge/盘锦辽东湾新区石化企业.xls', '盘锦辽东湾新区')
#main('D:/data/qichacha/日照高新区/日照高新区石化企业.xls', '日照高新区')
#main('D:/data/qichacha/日照东港区/merge/日照东港区石化企业.xls', '日照东港区')
#main('D:/data/qichacha/沈阳近海经济区/merge/沈阳近海经济区石化企业.xls', '沈阳近海经济区')
#main('D:/data/qichacha/天津唐屯/天津唐屯石化企业.xls', '天津唐屯')
#main('D:/data/qichacha/深圳前海合作区/merge/深圳前海合作区石化企业.xls', '深圳前海合作区')
main('D:/data/qichacha/洛阳自贸区/merge/洛阳自贸区石化企业.xls', '洛阳自贸区')

D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

洛阳恒基石化科技有限公司 -->库内无该企业


D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


洛阳创臻石化配件有限公司 -->库内无该企业
洛阳智昂石化工程技术有限公司 -->库内无该企业
洛阳三信石化设备有限公司 -->库内无该企业
河南河洛石化有限公司 -->库内无该企业
洛阳正铭石化工程有限公司 -->库内无该企业
洛阳兴罡石化设备有限公司 -->库内无该企业
洛阳标准石化有限公司 -->库内无该企业
洛阳捷安石化科技有限公司 -->库内无该企业
洛阳市旭龙石化科技有限公司 -->库内无该企业
河南左硕石化贸易有限公司 -->库内无该企业
洛阳市瓴越石化有限公司 -->库内无该企业
河南海疆石化科技有限公司 -->库内无该企业
中创光电微纳科技研究院(洛阳)有限公司 -->库内无该企业
洛阳优本精密轴承有限公司 -->库内无该企业
河南万海母海德机械科技有限公司 -->库内无该企业
洛阳沃赢实业有限公司 -->库内无该企业
洛阳海默流体设备有限公司 -->库内无该企业
洛阳圣斯仑密封件有限公司 -->库内无该企业
洛阳特浦环保设备有限公司 -->库内无该企业
河南增坤机械装备有限公司 -->库内无该企业
洛阳航宇机电设计有限公司 -->库内无该企业
洛阳公斌工贸有限公司 -->库内无该企业
洛阳徳得商贸有限公司 -->库内无该企业
河南育山智能科技有限公司 -->库内无该企业
河南沃赢石油股份有限公司 -->库内无该企业
洛阳普瑞曼自动控制技术有限公司 -->库内无该企业
河南火石文化传播有限公司 -->库内无该企业
洛阳科特信息科技有限公司 -->库内无该企业
洛阳科林石油化工科技有限公司 -->库内无该企业
河南豫金化工科技有限公司 -->库内无该企业
大象(洛阳)国际贸易有限公司 -->库内无该企业
洛阳中彭化工环保技术有限公司 -->库内无该企业
西拓(洛阳)矿业设计研究院有限公司 -->库内无该企业
福格森(洛阳)商贸有限公司 -->库内无该企业
诚拓(洛阳)新能源科技有限公司 -->库内无该企业
洛阳倍特进出口贸易有限公司 -->库内无该企业
清启(洛阳)智能检测科技有限公司 -->库内无该企业
河南旷瑞进出口贸易有限公司 -->库内无该企业
洛阳东泽进出口贸易有限公司 -->库内无该企业
洛阳沃旋进出口贸易有限公司 -->库内无该企业
洛阳汇千贸易有限公司 -->库内无该企业
洛阳嘉穗进出口贸易有限公司 -->库内无该企业
洛阳檀清进出口

河南育山智能科技有限公司 -->库内无该企业
河南沃赢石油股份有限公司 -->库内无该企业
洛阳普瑞曼自动控制技术有限公司 -->库内无该企业
河南火石文化传播有限公司 -->库内无该企业
洛阳科特信息科技有限公司 -->库内无该企业
洛阳科林石油化工科技有限公司 -->库内无该企业
河南豫金化工科技有限公司 -->库内无该企业
大象(洛阳)国际贸易有限公司 -->库内无该企业
洛阳中彭化工环保技术有限公司 -->库内无该企业
西拓(洛阳)矿业设计研究院有限公司 -->库内无该企业
福格森(洛阳)商贸有限公司 -->库内无该企业
诚拓(洛阳)新能源科技有限公司 -->库内无该企业
洛阳倍特进出口贸易有限公司 -->库内无该企业
清启(洛阳)智能检测科技有限公司 -->库内无该企业
河南旷瑞进出口贸易有限公司 -->库内无该企业
洛阳东泽进出口贸易有限公司 -->库内无该企业
洛阳沃旋进出口贸易有限公司 -->库内无该企业
洛阳汇千贸易有限公司 -->库内无该企业
洛阳嘉穗进出口贸易有限公司 -->库内无该企业
洛阳檀清进出口贸易有限公司 -->库内无该企业
洛阳麦熙进出口贸易有限公司 -->库内无该企业
洛阳法罗赛国际贸易有限公司 -->库内无该企业
洛阳新丰贸易有限公司 -->库内无该企业
河南转折点进出口贸易有限公司 -->库内无该企业
洛阳智诺贸易有限公司 -->库内无该企业
洛阳纽仕玛进出口贸易有限公司 -->库内无该企业
洛阳赫捷进出口贸易有限公司 -->库内无该企业
河南飞特轮矿业有限公司 -->库内无该企业
河南本通商贸有限公司 -->库内无该企业
洛阳连国商贸有限公司 -->库内无该企业
河南月硕机械设备有限公司 -->库内无该企业
洛阳国泰工贸有限公司 -->库内无该企业
洛阳昂泰商贸有限公司 -->库内无该企业
洛阳久润商贸有限公司 -->库内无该企业
洛阳善淼商贸有限公司 -->库内无该企业
洛阳辰亚劳务服务有限公司 -->库内无该企业
洛阳硕创轴承有限公司 -->库内无该企业
洛阳通言电子商务有限公司 -->库内无该企业
洛阳市万荣矿山机械设备有限公司 -->库内无该企业
洛阳朗云轴承有限公司 -->库内无该企业
洛阳盈旺商贸有限公司 -->库内无该企业
洛阳凯惠能源科技有限公司 -->库内无该企业
洛阳辰工商贸有限公司 -->库内无该企业


河南海疆石化科技有限公司 finished
洛阳新普石化设备开发有限公司 finished
洛阳奥凯石化科技有限责任公司 finished
洛阳石化工程设计有限公司自贸区分公司 finished
中创光电微纳科技研究院(洛阳)有限公司 finished
洛阳晟途机械有限公司 finished
洛阳优本精密轴承有限公司 finished
河南万海母海德机械科技有限公司 finished
洛阳沃赢实业有限公司 finished
洛阳佰洁清洗技术有限公司 finished
洛阳乾程环保技术服务有限公司 finished
洛阳琦昌节能科技有限公司 finished
洛阳康润石油化工科技开发有限公司 finished
洛阳驰路商贸有限公司 finished
洛阳海默流体设备有限公司 finished
洛阳圣斯仑密封件有限公司 finished
洛阳双英电气控制设备有限公司 finished
洛阳特浦环保设备有限公司 finished
河南增坤机械装备有限公司 finished
洛阳宁佳环保科技有限公司 finished
洛阳航宇机电设计有限公司 finished
河南民濮能源科技有限公司 finished
洛阳公斌工贸有限公司 finished
洛阳矿久机械设备有限公司 finished
洛阳徳得商贸有限公司 finished
洛阳常为先能源科技有限公司 finished
河南育山智能科技有限公司 finished
洛阳好利兰环保技术开发有限公司 finished
河南习风机电科技有限公司 finished
洛阳恒创重工科技有限公司 finished
洛阳高新博大环保新技术有限公司 finished
洛阳帝瑞实业有限公司 finished
洛阳友汇环保设备股份有限公司 finished
河南沃赢石油股份有限公司 finished
洛阳普瑞曼自动控制技术有限公司 finished
河南火石文化传播有限公司 finished
洛阳科特信息科技有限公司 finished
洛阳科林石油化工科技有限公司 finished
洛阳青石化学有限公司 finished
洛阳瑞昌环境工程有限公司 finished
洛阳源创电气有限公司 finished
洛阳圣瑞智能机器人有限公司 finished
中聚(河南)化工科技有限公司 finished
河南贵醇化工材料有限公司 finished
洛阳市

洛阳安之诺商贸有限公司 finished
河南展尚实业有限公司 finished
河南途达能源有限公司 finished
洛阳陛安迩机械科技有限公司 finished
洛阳财森新能源科技有限公司 finished
洛阳轩宁机电科技有限公司 finished
河南吉森智能家居科技有限公司 finished
洛阳中际生物科技有限公司 finished
河南省宽途建筑科技有限公司 finished
洛阳业程商贸有限公司 finished
洛阳思茂商贸有限公司 finished
洛阳蓝新生物科技有限公司 finished
河南洮崮商贸有限公司 finished
洛阳健阳科技有限公司 finished
洛阳北工智能科技有限公司 finished
洛阳泰源科技有限公司 finished
洛阳吉恩特生物科技有限公司 finished
河南润都电子科技有限公司 finished
洛阳菁洲纳米科技有限公司 finished
洛阳淦捷商贸有限公司 finished
洛阳中谷科贸有限公司 finished
洛阳百克特工贸有限公司 finished
河南普客电子设备有限公司 finished
洛阳誉芯金刚石有限公司 finished
洛阳林品商贸有限公司 finished
洛阳盛嘉新材料有限公司 finished
河南金厦商贸有限公司 finished
洛阳力肯航空科技有限公司 finished
洛阳乐担科技有限公司 finished
洛阳瑞岛干燥工程有限公司 finished
洛阳匠林商贸有限公司 finished
洛阳雅苑景观绿化工程有限公司 finished
洛阳易道商贸有限公司 finished
洛阳中创重型机械有限公司 finished
洛阳浙源园林绿化工程有限公司 finished
河南华颂实业有限公司 finished
洛阳果园工社文化传媒有限公司 finished
洛阳林轩园林绿化工程有限公司 finished
洛阳卓丽园林绿化工程有限公司 finished
洛阳丰蕾园林绿化工程有限公司 finished
河南省鑫融基园林绿化工程有限公司 finished
洛阳璟腾园林绿化工程有限公司 finished
洛阳栾绍园林绿化工程有限公司 finished
洛阳锐腾机械设备有限公司 finished
洛阳汇景园林绿化工程有限公司 finished
河南正涛建筑工程有限公司 fin

In [6]:
df_q = get_qichacha_df('D:/data/qichacha/舟山保税区/merge/舟山保税区石化企业.xls', '舟山保税区')
df_u = get_update_df(df_q)
df_u.head()

D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

舟山市浙东石化设备有限公司舟山港综合保税区分公司 -->库内无该企业


D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


浙江舟山米粒石化有限公司 -->库内无该企业
浙江舟山华力石化有限公司 -->库内无该企业
浙江舟山中经贸石化有限公司 -->库内无该企业
浙江舟山章贡石化有限公司 -->库内无该企业
浙江舟山沿海石化有限公司 -->库内无该企业
浙江舟山中宇石化有限公司 -->库内无该企业
浙江舟山隆庆石化有限公司 -->库内无该企业
舟山华油石化有限公司 -->库内无该企业
浙江自贸区峰峰石化有限公司 -->库内无该企业
舟山国储石化有限公司 -->库内无该企业
浙江舟山红鲁苏石化有限公司 -->库内无该企业
浙江自贸区克利石化有限公司 -->库内无该企业
舟山恒邦石化有限公司 -->库内无该企业
国祯石化(舟山)有限公司 -->库内无该企业
浙江自贸区渝舟石化有限公司 -->库内无该企业
浙江舟山广鑫石化有限公司 -->库内无该企业
舟山金隆石化有限公司 -->库内无该企业
舟山市周六石化有限公司 -->库内无该企业
浙江自贸区胜邦石化有限公司 -->库内无该企业
中海外天安(舟山)石化有限公司 -->库内无该企业
浙江自贸区盖亚石化有限公司 -->库内无该企业
浙江舟山鸿锐达石化有限公司 -->库内无该企业
浙江舟山林炎石化有限公司 -->库内无该企业
浙江舟山顺意石化有限公司 -->库内无该企业
浙江舟山甬金石化有限公司 -->库内无该企业
浙江舟山一诺石化有限公司 -->库内无该企业
浙江舟山甬镇石化有限公司 -->库内无该企业
浙江舟山国化石化有限公司 -->库内无该企业
浙江舟山万士宜石化有限公司 -->库内无该企业
浙江舟山马源石化有限公司 -->库内无该企业
浙江舟山晟宏石化有限公司 -->库内无该企业
浙江舟山萧恒石化有限公司 -->库内无该企业
浙江舟山伟峻石化有限公司 -->库内无该企业
舟山市宏兴石化有限公司 -->库内无该企业
舟山港综合保税区中能石油化工有限公司 -->库内无该企业
浙江舟山晟邦石化有限公司 -->库内无该企业
浙江舟山君鼎石化有限公司 -->库内无该企业
浙江舟山永晟石化有限公司 -->库内无该企业
浙江自贸区瑞信泰石化有限公司 -->库内无该企业
浙江舟山旺利石化有限公司 -->库内无该企业
浙江舟山广丰溢石化有限公司 -->库内无该企业
浙江舟山澜海石化有限公司 -->库内无该企业
麒润(舟山)石化有限公司 -->库内无该企业


舟山喆威石油化工有限公司 -->库内无该企业
舟山恒有石油化工有限公司 -->库内无该企业
舟山锦潮石油化工有限公司 -->库内无该企业
舟山成易石油化工有限公司 -->库内无该企业
舟山昌和石油化工有限公司 -->库内无该企业
舟山隆通石油化工有限公司 -->库内无该企业
舟山澎湃石油化工有限公司 -->库内无该企业
舟山长汇石油化工有限公司 -->库内无该企业
舟山万丰达石油化工有限公司 -->库内无该企业
舟山京港石油化工有限公司 -->库内无该企业
舟山管海石油化工有限公司 -->库内无该企业
舟山云返石油化工有限公司 -->库内无该企业
舟山金昌兴石油化工有限公司 -->库内无该企业
舟山澳元石油化工有限公司 -->库内无该企业
舟山海汇石油化工有限公司 -->库内无该企业
舟山甬恒石油化工有限公司 -->库内无该企业
舟山逸帆石油化工有限公司 -->库内无该企业
中电石油化工(舟山)有限公司 -->库内无该企业
舟山楚泽石油化工有限公司 -->库内无该企业
舟山赵氏石油化工有限公司 -->库内无该企业
舟山尊孚石油化工有限公司 -->库内无该企业
舟山中油金文石油化工有限公司 -->库内无该企业
舟山义泓石油化工有限公司 -->库内无该企业
舟山海晟石油化工有限公司 -->库内无该企业
舟山润辰石油化工有限公司 -->库内无该企业
舟山思泽石油化工有限公司 -->库内无该企业
港鑫(舟山)石油化工有限公司 -->库内无该企业
华瀛(舟山)石油化工有限公司 -->库内无该企业
中油晟文石油化工(舟山)有限公司 -->库内无该企业
中硕石油化工(舟山)有限公司 -->库内无该企业
舟山顺安发石油化工有限公司 -->库内无该企业
和利石油化工(舟山)有限公司 -->库内无该企业
浙江心润石油化工有限公司 -->库内无该企业
舟山鸿泽石油化工有限公司 -->库内无该企业
运安(舟山)石油化工有限公司 -->库内无该企业
舟山北融石油化工有限公司 -->库内无该企业
舟山鑫胜石油化工有限公司 -->库内无该企业
浙江意利石油化工有限公司 -->库内无该企业
浙江纪良石油化工有限公司 -->库内无该企业
舟山玺华石油化工有限公司 -->库内无该企业
舟山中恒宇石油化工有限公司 -->库内无该企业
中储北方石油化工有限公司 -->库内无该企业
舟山市黔静石化有限公司 

...

舟山市仟亿正通化工有限公司 -->库内无该企业
舟山港综合保税区兴琪物流有限公司 -->库内无该企业
浙江自贸区车德化工原料有限公司 -->库内无该企业
浙江金庚化工有限公司 -->库内无该企业
舟山荣源化工有限公司 -->库内无该企业
浙江舟山唯美能源化工有限公司 -->库内无该企业
舟山港综合保税区朝润进出口有限公司 -->库内无该企业
舟山市安楠化工有限公司 -->库内无该企业
浙江自贸区启帆化工有限公司 -->库内无该企业
浙江自贸区保禄化工有限公司 -->库内无该企业
舟山博荣化工有限公司 -->库内无该企业
浙江自贸区博森化工贸易有限公司 -->库内无该企业
浙江舟山确客能源有限公司 -->库内无该企业
浙江舟山华畅能源有限公司 -->库内无该企业
浙江舟山东玉能源有限公司 -->库内无该企业
舟山港贸物流有限公司 -->库内无该企业
浙江舟山粤创顺能源有限公司 -->库内无该企业
浙江舟山天浩能源有限公司 -->库内无该企业
浙江舟山七江能源有限公司 -->库内无该企业
浙江舟山中龙能源有限公司 -->库内无该企业
浙江舟山驰邦能源有限公司 -->库内无该企业
浙江舟山亿翔企业发展有限公司 -->库内无该企业
中锦能源(浙江舟山)有限公司 -->库内无该企业
浙江舟山唯达能源有限公司 -->库内无该企业
浙江舟山前景能源有限公司 -->库内无该企业
浙江舟山赞宏能源有限公司 -->库内无该企业
浙江舟山浩客贸易有限公司 -->库内无该企业
浙江舟山天德能源有限公司 -->库内无该企业
浙江舟山聚川能源有限公司 -->库内无该企业
浙江舟山星曜能源有限公司 -->库内无该企业
浙江舟山海仕威能源有限公司 -->库内无该企业
浙江舟山德佳能源有限公司 -->库内无该企业
浙江舟山聚鑫能源有限公司 -->库内无该企业
浙江舟山铭丰能源有限公司 -->库内无该企业
浙江舟山裕众能源有限公司 -->库内无该企业
浙江舟山油咖能源科技有限公司 -->库内无该企业
舟山市赋能物流有限公司 -->库内无该企业
舟山市长冠石油贸易有限公司 -->库内无该企业
浙江舟山立寰能源有限公司 -->库内无该企业
浙江舟山华贵国源能源有限公司 -->库内无该企业
浙江舟山宁鲲能源有限公司 -->库内无该企业
浙江舟山铭淐源石油有限公司 -->库内无该企业
舟山市明笙物资有限责任公

舟山澳威能源有限公司 -->库内无该企业
浙江自贸区恒奥石油有限公司 -->库内无该企业
骏汇能源(浙江)有限公司 -->库内无该企业
浙广石油(舟山)有限公司 -->库内无该企业
浙江自贸区京华能源有限公司 -->库内无该企业
浙江舟山京元达能源有限公司 -->库内无该企业
扬子江能源(浙江自贸区)有限公司 -->库内无该企业
舟山泽创石油有限公司 -->库内无该企业
青唐能源(舟山)有限公司 -->库内无该企业
浙江自贸区科缘能源有限公司 -->库内无该企业
浙江自贸区博赢能源有限公司 -->库内无该企业
浙江自贸区宁润能源有限公司 -->库内无该企业
舟山海青能源有限公司 -->库内无该企业
浙江自贸区铂誉能源有限公司 -->库内无该企业
浙江自贸区瑞合能源有限公司 -->库内无该企业
中冀能源(舟山)有限公司 -->库内无该企业
浙江自贸区东创能源有限公司 -->库内无该企业
浙江自贸区蓝德能源有限公司 -->库内无该企业
浙江自贸区晟诺蓝能源有限公司 -->库内无该企业
睿喜能源(舟山)有限公司 -->库内无该企业
春田能源(浙江自贸区)有限公司 -->库内无该企业
浙江自贸区稳程能源有限公司 -->库内无该企业
舟山弘昌能源有限公司 -->库内无该企业
浙江自贸区聚尚能源有限公司 -->库内无该企业
浙江舟山信安能源有限公司 -->库内无该企业
亿企团供应链(舟山)有限公司 -->库内无该企业
舟山旭鑫能源有限公司 -->库内无该企业
浙江自贸区景渱能源有限公司 -->库内无该企业
浙江舟山阳亮能源有限公司 -->库内无该企业
舟山泽康文化传媒工作室 -->库内无该企业
舟山尚岛商贸有限公司 -->库内无该企业
信力石油(舟山)有限公司 -->库内无该企业
舟山市奥英石油技术服务有限公司 -->库内无该企业
舟山禾顺石油贸易有限公司 -->库内无该企业
浙江云凯石油有限公司 -->库内无该企业
浙江舟山豪兴能源有限公司 -->库内无该企业
舟山市银鼎能源有限公司 -->库内无该企业
舟山市骏霆能源有限公司 -->库内无该企业
舟山市天汇物流有限公司 -->库内无该企业
浙江自贸区星升能源有限公司 -->库内无该企业
浙江自贸区亿锦能源有限公司 -->库内无该企业
浙江自贸区国赋能源有限公司 -->库内无该企业
浙江自贸区锦弘能源有限公司 -->库内无该企

nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
nan -->库内无该企业
6302 13964398623
nan -->库内无该企业
6310 13672096631
nan -->库内无该企业
6313 18899180562
nan -->库内无该企业
6321 13957786909
nan -->库内无该企业
6336 15757931257
nan -->库内无该企业
6342 15658332858
nan -->库内无该企业
6344 18268763185
nan -->库内无该企业
6350 18173586888
nan -->库内无该企业
6354 18666809269
nan -->库内无该企业
6355 15356472832
nan -->库内无该企业
6356 18354696009
nan -->库内无该企业
6358 18258849667
nan -->库内无该企业
6363 13817616188
nan -->库内无该企业
6365 18576670314
nan -->库内无该企业
6383 13699175440
nan -->库内无该企业
638

nan -->库内无该企业
7575 13788898105
nan -->库内无该企业
7583 13849318808
nan -->库内无该企业
7594 15996315068
nan -->库内无该企业
7600 13336880000
nan -->库内无该企业
7602 13868220771
nan -->库内无该企业
7608 15552733788
nan -->库内无该企业
7613 15166574222
nan -->库内无该企业
7616 15106347888
nan -->库内无该企业
7617 13193722222
nan -->库内无该企业
7629 13816771922
nan -->库内无该企业
7630 13858191677
nan -->库内无该企业
7645 13683963316
nan -->库内无该企业
7648 15381079828
nan -->库内无该企业
7650 18116225568
nan -->库内无该企业
7657 13646509786
nan -->库内无该企业
7658 15888524716
nan -->库内无该企业
7671 13818389774
nan -->库内无该企业
7674 13032216754
nan -->库内无该企业
7682 13052471577
nan -->库内无该企业
7684 13616476616
nan -->库内无该企业
7702 15957078063
nan -->库内无该企业
7703 13505882520
nan -->库内无该企业
7737 13967205211
nan -->库内无该企业
7744 13803545678
nan -->库内无该企业
7751 18653332939
nan -->库内无该企业
7775 13912436168
nan -->库内无该企业
7776 15931821263
nan -->库内无该企业
7780 13949301420
nan -->库内无该企业
7800 13968596133
nan -->库内无该企业
7807 15900862678
nan -->库内无该企业
7837 18233710775
nan -->库内无该企业
7862 15588681333
nan -->库

,com,province,city,address,phone,province_real,city_real,area,q_id
0,浙江舟山市中擎石化有限公司,浙江,舟山市,浙江省舟山市定海区舟山港综合保税区企业服务中心301-11742室(自贸试验区内),None,None,None,舟山保税区,None
1,浙江舟山市澳贺石化有限公司,浙江,舟山市,浙江省舟山市定海区舟山港综合保税区企业服务中心301-13171室(自贸试验区内),None,None,None,舟山保税区,None
2,舟山市浙东石化设备有限公司舟山港综合保税区分公司,浙江,舟山市,浙江省舟山市定海区舟山港综合保税区企业服务中心303-8684室(自贸试验区内),13967206551,浙江,舟山市,舟山保税区,None
3,舟山港综合保税区能源化工有限公司,浙江,舟山市,浙江省舟山市定海区舟山经济开发区新港工业园区4号楼105-33室(自贸试验区内),None,None,None,舟山保税区,None
4,浙江舟山元翔顺吉石化有限公司,浙江,舟山市,浙江省舟山市定海区舟山港综合保税区企业服务中心301-28067室(自贸试验区内),None,None,None,舟山保税区,None


In [7]:
df_u.tail()

,com,province,city,address,phone,province_real,city_real,area,q_id
8195,NaN,NaN,市区,浙江省舟山市定海区舟山港综合保税区企业服务中心301-12972室(自贸试验区内),None,None,None,舟山保税区,None
8196,NaN,NaN,市区,中国(浙江)自由贸易试验区舟山市定海区舟山港综合保税区企业服务中心301-13314室,None,None,None,舟山保税区,None
8197,NaN,NaN,市区,浙江省舟山市定海区舟山港综合保税区企业服务中心301-28149室(自贸试验区内),None,None,None,舟山保税区,None
8198,NaN,NaN,市区,浙江省舟山市定海区舟山港综合保税区企业服务中心301-12479室(自贸试验区内),None,None,None,舟山保税区,None
8199,NaN,NaN,市区,浙江省舟山市定海区舟山港综合保税区企业服务中心301-993室(自贸试验区内),None,None,None,舟山保税区,None


In [63]:
#df_u.head()
update_df(df_u)

大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished
大连濠嘉石化有限公司 finished


KeyboardInterrupt: 

In [40]:
df = df_u.copy()
#df.isnull().any(axis=1)
phone = df.loc[4,'phone']
if np.isnan(phone):
    df.loc[4, 'phone'] = 0
df.head()

,com,province,city,address,phone,province_real,city_real,area,q_id
0,大连濠嘉石化有限公司,辽宁,大连市,辽宁省大连保税区海天路慧能大厦33号13层1301A-15,None,None,None,大连保税区,None
1,大连欣源石化有限公司,辽宁,大连市,辽宁省大连保税区慧能大厦906B-1-9,0411-87078888,None,None,大连保税区,None
2,大连豪达石化有限公司,辽宁,大连市,辽宁省大连保税区慧能大厦507C,15040426940,辽宁,大连市,大连保税区,None
3,大连兆博怡石化能源有限公司,辽宁,大连市,辽宁省大连保税区慧能大厦160A,13309860012,辽宁,大连市,大连保税区,None
4,中能国联石化(大连)有限公司,辽宁,大连市,辽宁省大连保税区海天路24号慧能大厦33号13层1304D-19,0,None,None,大连保税区,None


In [55]:
df = df_u.copy()
df['phone'].fillna('-', inplace=True)
df.head()

,com,province,city,address,phone,province_real,city_real,area,q_id
0,大连濠嘉石化有限公司,辽宁,大连市,辽宁省大连保税区海天路慧能大厦33号13层1301A-15,-,None,None,大连保税区,None
1,大连欣源石化有限公司,辽宁,大连市,辽宁省大连保税区慧能大厦906B-1-9,0411-87078888,None,None,大连保税区,None
2,大连豪达石化有限公司,辽宁,大连市,辽宁省大连保税区慧能大厦507C,15040426940,辽宁,大连市,大连保税区,None
3,大连兆博怡石化能源有限公司,辽宁,大连市,辽宁省大连保税区慧能大厦160A,13309860012,辽宁,大连市,大连保税区,None
4,中能国联石化(大连)有限公司,辽宁,大连市,辽宁省大连保税区海天路24号慧能大厦33号13层1304D-19,-,None,None,大连保税区,None


In [20]:
p = df_u.loc[0, 'phone']
np.isnan(p)

True

In [44]:
1>None

TypeError: '>' not supported between instances of 'int' and 'NoneType'

In [1]:
#对企查查整合后的数据进行批量导入
import pandas

dir_qichacha = "D:/data/qichacha/大连保税区/merge/保税区石化企业.xls"

df = pandas.read_excel(dir_qichacha)
df_new = df[['企业名称','省份','城市','地址','电话']]
df_new.columns = ['com','province','city','address','phone']
df_new['province_real'] = None
df_new['city_real'] = None
df_new['area'] = '大连保税区'
df_new['q_id'] = None
df_new.head()

D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
D:\Selvaria\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

,com,province,city,address,phone,province_real,city_real,area,q_id
0,大连濠嘉石化有限公司,辽宁,大连市,辽宁省大连保税区海天路慧能大厦33号13层1301A-15,NaN,None,None,大连保税区,None
1,大连欣源石化有限公司,辽宁,大连市,辽宁省大连保税区慧能大厦906B-1-9,0411-87078888,None,None,大连保税区,None
2,大连豪达石化有限公司,辽宁,大连市,辽宁省大连保税区慧能大厦507C,15040426940,None,None,大连保税区,None
3,大连兆博怡石化能源有限公司,辽宁,大连市,辽宁省大连保税区慧能大厦160A,13309860012,None,None,大连保税区,None
4,中能国联石化(大连)有限公司,辽宁,大连市,辽宁省大连保税区海天路24号慧能大厦33号13层1304D-19,NaN,None,None,大连保税区,None


In [2]:
row = list(df_new.loc[0].values)
row

['大连濠嘉石化有限公司',
 '辽宁',
 '大连市',
 '辽宁省大连保税区海天路慧能大厦33号13层1301A-15',
 nan,
 None,
 None,
 '大连保税区',
 None]